In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:02<00:00, 4359007.39it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 128688.96it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:03<00:00, 525950.66it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 3114358.14it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [3]:
# 2. Define the neural network architecture
class InceptionModule(nn.Module):
    def __init__(self, in_channels):
        super(InceptionModule, self).__init__()

        # 1x1 convolution branch
        self.branch1x1 = nn.Conv2d(in_channels, 64, kernel_size=1)

        # 3x3 convolution branch (followed by a 1x1 convolution)
        self.branch3x3 = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=1),
            nn.Conv2d(64, 64, kernel_size=3, padding=1)
        )

        # 5x5 convolution branch (followed by a 1x1 convolution)
        self.branch5x5 = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=1),
            nn.Conv2d(64, 64, kernel_size=5, padding=2)
        )

        # 3x3 max pooling branch (followed by a 1x1 convolution)
        self.branch_pool = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, 64, kernel_size=1)
        )

    def forward(self, x):
        branch1x1 = self.branch1x1(x)
        branch3x3 = self.branch3x3(x)
        branch5x5 = self.branch5x5(x)
        branch_pool = self.branch_pool(x)

        # Concatenating along the channel dimension
        outputs = [branch1x1, branch3x3, branch5x5, branch_pool]
        return torch.cat(outputs, 1)



class InceptionNet(nn.Module):
    def __init__(self):
        super(InceptionNet, self).__init__()

        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        self.inception = InceptionModule(in_channels=64)
        self.conv2 = nn.Conv2d(256, 128, kernel_size=3, padding=1)
        self.fc = nn.Linear(128 * 7 * 7, 10)  # Input image size is 28x28 (MNIST)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.inception(x)
        x = F.relu(self.conv2(x))
        x = F.adaptive_avg_pool2d(x, (7, 7))
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x



In [4]:
# 3. Initialize the network, loss function, and optimizer
model = InceptionNet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print(model)

InceptionNet(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (inception): InceptionModule(
    (branch1x1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (branch3x3): Sequential(
      (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (branch5x5): Sequential(
      (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    )
    (branch_pool): Sequential(
      (0): MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
      (1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    )
  )
  (conv2): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc): Linear(in_features=6272, out_features=10, bias=True)
)


In [5]:
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()  # Set the model to training mode
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()  # Zero the gradients
        output = model(data)   # Forward pass
        loss = criterion(output, target)  # Compute the loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')


In [6]:
def test(model, device, test_loader, criterion):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0
    correct = 0
    with torch.no_grad():  # No need to compute gradients during evaluation
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # Sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)\n')


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(1, 5):
    train(model, device, train_loader, optimizer, criterion, epoch)
    test(model, device, test_loader, criterion)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.303787
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.203742
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.032882
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.180824
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.022528
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.126758
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.073993
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.257571
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.083299
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.142185

Test set: Average loss: 0.0010, Accuracy: 9826/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.141677
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.047995
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.089488
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.019702
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.007767
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.030970
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.056256
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.013558
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.047971
T